<i>Licensed under the MIT License.</i>

# Deployment of a classification model to an Azure app service


## Table of contents

1. [Introduction](#intro)
1. [Model retrieval](#model)
1. [Model deployment](#deploy)
  1. [Workspace retrieval](#workspace)
  1. [Model registration](#register)
  1. [Scoring script](#scoring)
  1. [Environment setup](#env)
  1. [Packaging model for Azure app](#package)
  1. [Deploying the web app](#app)
1. [Testing the web app](#test)

## 1. Introduction <a id="intro"></a>

This notebook is similar to notebook 21 in the classification scenario, but instead of deploying our model to a webservice in the AzureML resource group, we will deploy it to an app service. While an AzureML deployment is satisfactory for sharing your model, it cannot interfaced with via a web application. Azure app service enables this by allowing us to set up our own security policies, particularly CORS policies that allow us to make requests to the model from other sources.

We will:
- Register a model
- Create an environment that contains our model
- Deploy an app service using this environment on [Azure Container Instances](https://azure.microsoft.com/en-us/services/container-instances/)

This notebook follows the instructions the article [Deploy a machine learning model to Azure App Service](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-app-service#deploy-image-as-a-web-app).

### Pre-requisites <a id="pre-reqs"></a>
Like the other notebook, an Azure workspace is required for this notebook. If we don't have one, we need to first run through the short 20_azure_workspace_setup.ipynb notebook in the classification scenario to create it.

While it is not required that you have run notebook 21_deployment_on_azure_container_instances.ipynb from the classification scenario, as this notebook repeats the necessary work from there, it is still recommended that you have run and understood that notebook. We will only be explaining the work specific to deploying to an app service here.

Additionally, you will need to have installed [The Azure CLI](https://docs.microsoft.com/cli/azure/install-azure-cli?view=azure-cli-latest).

### Library import <a id="libraries"></a>
Throughout this notebook, we will be using a variety of libraries. We are listing them here for better readibility.

In [11]:
# For automatic reloading of modified libraries
%reload_ext autoreload
%autoreload 2

# Regular python libraries
import os
import sys

# fast.ai
from fastai.vision import models

# Azure
import azureml.core
from azureml.core import Experiment, Workspace
from azureml.core.image import ContainerImage
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import WebserviceException

# Computer Vision repository
sys.path.extend([".", "../..", "../../.."])
# This "sys.path.extend()" statement allows us to move up the directory hierarchy 
# and access the utils_cv package
from utils_cv.common.deployment import generate_yaml
from utils_cv.common.data import root_path 
from utils_cv.classification.model import IMAGENET_IM_SIZE, model_to_learner

# Check core SDK version number
print(f"Azure ML SDK Version: {azureml.core.VERSION}")

Azure ML SDK Version: 1.2.0


## 2. Model retrieval and export <a id="model"></a>

In [ ]:
learn = model_to_learner(models.resnet18(pretrained=True), IMAGENET_IM_SIZE)

output_folder = os.path.join(os.getcwd(), 'outputs')
model_name = 'im_classif_resnet18'  # Name we will give our model both locally and on Azure
pickled_model_name = f'{model_name}.pkl'
os.makedirs(output_folder, exist_ok=True)

learn.export(os.path.join(output_folder, pickled_model_name))

## 3. Model deployment on Azure <a id="deploy"></a>

As with other deployment notebooks, you will need to supply your own subcription id, resource group, workspace name, and workspace region for this notebook.

In [ ]:
subscription_id = "YOUR_SUBSCRIPTION_ID_HERE"
resource_group = "YOUR_RESOURCE_GROUP_HERE"  
workspace_name = "YOUR_WORKSPACE_NAME_HERE"  
workspace_region = "YOUR_WORKSPACE_REGION_HERE"

### 3.A Workspace retrieval <a id="workspace"></a>

In [ ]:
from utils_cv.common.azureml import get_or_create_workspace

ws = get_or_create_workspace(
        subscription_id,
        resource_group,
        workspace_name,
        workspace_region)

### 3.B Model registration  (Without experiment) <a id="register"></a>

In [ ]:
model = Model.register(
    model_path = os.path.join('outputs', pickled_model_name),
    model_name = model_name,
    tags = {"Model": "Pretrained ResNet18"},
    description = "Image classifier",
    workspace = ws
)

### 3.C Scoring script <a id="scoring"></a>

In [2]:
scoring_script = "score.py"

%%writefile $scoring_script
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license.

import os
import json

from base64 import b64decode
from io import BytesIO
from azureml.contrib.services.aml_response import AMLResponse

from azureml.core.model import Model
from fastai.vision import load_learner, open_image

def init():
    global model
    model_path = Model.get_model_path(model_name='im_classif_resnet18')
    # ! We cannot use the *model_name* variable here otherwise the execution on Azure will fail !

    model_dir_path, model_filename = os.path.split(model_path)
    model = load_learner(model_dir_path, model_filename)


def run(raw_data):

    # Expects raw_data to be a list within a json file
    result = []
    
    for im_string in json.loads(raw_data)['data']:
        im_bytes = b64decode(im_string)
        try:
            im = open_image(BytesIO(im_bytes))
            pred_class, pred_idx, outputs = model.predict(im)
            result.append({"label": str(pred_class), "probability": str(outputs[pred_idx].item())})
        except Exception as e:
            result = AMLResponse({"label": str(e), "probability": ""})
            #result.append({"label": str(e), "probability": ''})
    return result

### 3.D Environment setup <a id="env"></a>
#### Using `azureml.core.environment` to build the docker image and for deployment

In [ ]:
# Create a deployment-specific yaml file from classification/environment.yml
try:
    generate_yaml(
        directory=os.path.join(root_path()), 
        ref_filename='environment.yml',
        needed_libraries=['pytorch', 'spacy', 'fastai', 'dataclasses'],
        conda_filename='myenv.yml'
    )
    # Note: Take a look at the generate_yaml() function for details on how to create your yaml file from scratch

except FileNotFoundError:
    raise FileNotFoundError("The *environment.yml* file is missing - Please make sure to retrieve it from the github repository")
    

from azureml.core import Environment
from azureml.core.environment import DEFAULT_CPU_IMAGE

cv_test_env = Environment.from_conda_specification(name="im_classif_resnet18", file_path="myenv.yml")

# required to have required inferencing packages preinstalled in the resulting docker image
cv_test_env.inferencing_stack_version="latest"

# let's use the default CPU image and add a few required packages
cv_test_env.docker.base_dockerfile="""FROM {}
RUN apt-get update && \
    apt-get install -y libssl-dev build-essential libgl1-mesa-glx
""".format(DEFAULT_CPU_IMAGE)

# setting docker.base_image to None to use the base_dockerfile to build the image
cv_test_env.docker.base_image=None

# Now, let's try registering the environment. You'll be able to see the specified environment.
cv_test_env.register(ws)

# Since building the docker image for the first time requires a few minutes, let's start building the image
# that we'll be using for deployment now as we'll be able to monitor the build through the streamed log.
cv_test_env.build(ws).wait_for_completion(show_output=True)

## 3.E Packaging the model for use in Azure app service <a id="package"></a>

Now we break off from `21_deployment_on_azure_container_instances.ipynb`, as we need to create a package for our model that we will use to deploy an Azure app service. First, we set up an inference configuration that encapsulates information regarding the model's dependencies and entry script.

Next, using [Model.package](https://docs.microsoft.com//python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#package-workspace--models--inference-config-none--generate-dockerfile-false-), we create a Docker image that contains the workspace we are deploying to, the model(s) we are using, and any dependencies those models might have.

We print out the package location, as we will need this below in order to create and deploy the app.

In [ ]:
from azureml.core.model import InferenceConfig

# A configuration containing our entry script and the dependencies required by the model
inference_config = InferenceConfig(entry_script='score.py', environment=cv_test_env)

# Create a ModelPackage object containing information about our workspace, model, and configuration
package = Model.package(ws, [model], inference_config)
package.wait_for_creation(show_output=True)
print("Package location:\n", package.location)

## 3.F Deploying the web app <a id="app"></a>

Finally, we are ready to deploy our model as an Azure app service. This process will be done using the Azure CLI.

#### Creating a resource group and app service plan

You can skip this part if you have already created a resource group and service plan for your app. Otherwise, we will need to create them here. You should replace `<myresourcegroup>` and `<myplanname>` with the names you wish to use. Also replace `"LOCATION"` with the region you would like to use for your app.

```bash
    az group create --name <myresourcegroup> --location "LOCATION"
    az appservice plan create --name <myplanname> --resource-group <myresourcegroup> --sku B1 --is-linux
```

#### Creating the web app

Now we create the webapp. You should replace `<myresourcegroup>` and `<myplanname>` with the same names you used when you created your resource group and app service plan. Additionally, you should replace `<packagelocation>` with the package location returned by the python script.

```bash
    az webapp create --resource-group <myresourcegroup> --plan myplanname --name <app-name> --deployment-container-image-name <packagelocation>
```
    
This will return a JSON response containing information about our app.

#### Retrieve login credentials for Azure Container Registry

As part of the process in creating the image for our app, we uploaded it to the Azure Container Registry (ACR). We will need the login credentials for this image in the ACR in order to activate our app. When we printed out the location of the package above, it gave us an output similar to 

```bash
    <myacr>.azurecr.io/<imagename>
```

Replace `<myacr>` below with `<myacr>` from the package location.

```bash
    az acr credential show --name <myacr>
```
    
This should give a JSON response similar to the one below:

```bash
    {
    "passwords": [
        {
        "name": "password",
        "value": "Iv0lRZQ9762LUJrFiffo3P4sWgk4q+nW"
        },
        {
        "name": "password2",
        "value": "=pKCxHatX96jeoYBWZLsPR6opszr==mg"
        }
    ],
    "username": "myml08024f78fd10"
    }
```


#### Activating the web app

At this point, our web app has been created, but it is not active. In order to activate it, we need to provide the credentials to the ACR that holds the app's image.

Replace `<myresourcegroup>`, `<packagename>`, and `<myacr>` as we have above. You should also replace `<app-name>` with the name you would like to give the app. Then, replace `<username>` and `<password>` with the credentials we retrieved above.

```bash
    az webapp config container set --name <app-name> --resource-group <myresourcegroup> --docker-custom-image-name <packagename> --docker-registry-server-url <myacr>.azurecr.io --docker-registry-server-user <username> --docker-registry-server-password <password>
```
    
If the app name you chose is unavailable, you will need to provide a different name.

After running this command, it will return a JSON response similar to the one below:

```bash
    [
    {
        "name": "WEBSITES_ENABLE_APP_SERVICE_STORAGE",
        "slotSetting": false,
        "value": "false"
    },
    {
        "name": "DOCKER_REGISTRY_SERVER_URL",
        "slotSetting": false,
        "value": "https:<myacr>.azurecr.io"
    },
    {
        "name": "DOCKER_REGISTRY_SERVER_USERNAME",
        "slotSetting": false,
        "value": "myml08024f78fd10"
    },
    {
        "name": "DOCKER_REGISTRY_SERVER_PASSWORD",
        "slotSetting": false,
        "value": null
    },
    {
        "name": "DOCKER_CUSTOM_IMAGE_NAME",
        "value": "DOCKER|<packagelocation>"
    }
    ]
```

Now our app will begin loading the image from ACR. This may take awhile. We can monitor its progress using the log stream tab in the Azure portal.

Once the app is deployed, you can find the hostname using:
```bash
    az webapp show --name <app-name> --resource-group <myresourcegroup>
```

#### Enabling cross origin resource sharing (CORS)

Lastly, we need to enable CORS, so our app can provide results to other websites that want to run the model. Replace `<app-name>` with your app's name, and `<new-origin>` with the origin you would like to allow resource sharing for (to enable CORS from all origins, you can enter `"*"`).
```bash
    az webapp cors add -n <app-name> --alowed-origins <new-origin>
```

## 4. Test the web app <a id="test"></a>

We can run the below code to test that our app is deployed and functional. Here we select two images from Microsoft image set, convert them into base64, and send a post request to our app. You should replace the service_uri string with your app's scoring uri.

In [10]:
from utils_cv.common.image import im2base64, ims2strlist
from utils_cv.common.data import data_path
import requests
import json

# Extract test images paths
im_url_root = "https://cvbp-secondary.z19.web.core.windows.net/images/"
im_filenames = ["cvbp_milk_bottle.jpg", "cvbp_water_bottle.jpg"]

for im_filename in im_filenames:
    # Retrieve test images from our storage blob
    r = requests.get(os.path.join(im_url_root, im_filename))

    # Copy test images to local data/ folder
    with open(os.path.join(data_path(), im_filename), 'wb') as f:
        f.write(r.content)

# Extract local path to test images
local_im_paths = [os.path.join(data_path(), im_filename) for im_filename in im_filenames]

# Convert images to json object
im_string_list = ims2strlist(local_im_paths)

service_uri = "YOUR_WEBAPP_URI_HERE"

payload = json.dumps({"data": im_string_list})
headers = {'Content-Type':'application/json'}

resp = requests.post(service_uri, payload, headers=headers)

print(resp.status_code)
print(resp.text)

200
[{"label": "water_bottle", "probability": "0.800184428691864"}, {"label": "water_bottle", "probability": "0.6857761144638062"}]
